In [9]:
import itertools

from extract import extract_dataset_archive
from split import convert_train_set_to_images
import os
from tensorflow.keras.layers import Dense, LSTM, Activation, Flatten, TimeDistributed, ConvLSTM2D, Conv3D, BatchNormalization
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale
# from tensorflow.keras.layers.convolutional import 
# from tensorflow.keras.layers.convolutional_recurrent import ConvLSTM2D
# from tensorflow.keras.layers.normalization import BatchNormalization
# os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml --checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ../dataset/NonViolence/NV_1.mp4 --outdir ../dataset --sp')

def prepare_dataset():
    extract_dataset_archive()
    convert_train_set_to_images()


def convert_videos_to_json(path_in, path_out):
    os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml '
              '--checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ' + path_in + ' --outdir ' + path_out + ' --sp')


#prepare_dataset()
#os.chdir("AlphaPose")

# for i in range(1, 1001):
#    file_in = str("../dataset/Violence/V_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/V_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)

#for i in range(1, 1001):
#    file_in = str("../dataset/NonViolence/NV_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/NV_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)



person_count = 5
max_frames = 280


In [ ]:
model = Sequential()
model.add(TimeDistributed(Flatten(input_shape=(5, 68))))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.build(input_shape=(280, 5, 68))
model.summary()

In [10]:
csv_dir = "dataset/csvs"

csv_files = os.listdir(csv_dir)

train_files, test_files = train_test_split(csv_files, train_size=0.8)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri multiple                  0         
_________________________________________________________________
lstm_2 (LSTM)                multiple                  67600     
_________________________________________________________________
dense_6 (Dense)              multiple                  101       
Total params: 67,701
Trainable params: 67,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train = []
y_train = []
trained = 0
for file_name in train_files:
    
#     if trained == 2000:
#         break
    trained += 1
    
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    current_frame = 0
    
    violent = 0
    
    if len(csv) > 0:
        violent = csv.violent[0]

        csv = csv.drop('violent',axis=1)

        labels = csv.columns.values

        for label in labels[2:]:
            minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
            csv[[label]] = minmax_scale.transform(csv[[label]])

        data = csv.to_numpy()

        val_cumulative = {}
        frame_ids = set()
        for person in data:
            frame_ids.add(person[0])
            frame, person_id = person[:2]

            vel_array = []
            for vel in range(5, 70, 4):
                vel_array.append(vel)

            total_vel = sum(vel_array)

            if person_id in val_cumulative:
                val_cumulative[person_id] += total_vel
            else:
                val_cumulative[person_id] = 0


        val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
        top_people = dict(itertools.islice(val_cumulative.items(), person_count))
        top_people_dataset = {}


        for frame_id in frame_ids:
            # Trim to 280 if above
            if current_frame == max_frames:
                break
            top_people_dataset[frame_id] = []
            people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
            fill_amount = person_count
            for person in people:
                top_people_dataset[frame_id].append(person[2:])
                fill_amount -= 1
            for i in range(0, fill_amount):
                top_people_dataset[frame_id].append(np.zeros(68))
            current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
    tmp = np.asarray(list(top_people_dataset.values()))
    
#     x_train.append(tmp)
    for entry in tmp:
        x_train.append(entry)

#     x_train = np.append(x_train, np.asarray([list(top_people_dataset.values())]), axis=0)
        
#     print(x_train.shape)
    
    if violent == 0:
        for entry in np.zeros(len(tmp)):
            y_train.append(entry)
    else:
        for entry in np.ones(len(tmp)):
            y_train.append(entry)
#         y_train.append(np.ones(len(x_train)))
    

In [17]:
x_train = np.asarray(list(x_train))
y_train = np.asarray(list(y_train))

model.fit(x_train, y_train, epochs=20, batch_size=100, validation_split=0.2)
scores = model.evaluate(x_train, y_train, verbose=1)

Train on 17920 samples, validate on 4480 samples
Epoch 1/20
17920/17920 [==============================] - 4s 221us/sample - loss: 0.4661 - accuracy: 0.7050 - val_loss: 1.0926 - val_accuracy: 0.5522
Epoch 2/20
17920/17920 [==============================] - 4s 211us/sample - loss: 0.4615 - accuracy: 0.7078 - val_loss: 1.0748 - val_accuracy: 0.5496
Epoch 3/20
17920/17920 [==============================] - 4s 222us/sample - loss: 0.4691 - accuracy: 0.7039 - val_loss: 1.0647 - val_accuracy: 0.5442
Epoch 4/20
 9700/17920 [===============>..............] - ETA: 1s - loss: 0.4578 - accuracy: 0.7073

KeyboardInterrupt: 

In [16]:
x_test = []
y_test = []

for file_name in test_files:
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    violent = csv.violent[0]
    
    # Resize all CSV files to same arbitrary limit
    csv = csv.drop('violent',axis=1)
    
    labels = csv.columns.values
    
    
    for label in labels[2:]:
        minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
        csv[[label]] = minmax_scale.transform(csv[[label]])
    
#     column_trans = ColumnTransformer(
#         [('scaler', MinMaxScaler(),list(range(2,70)))],
#         remainder='passthrough') 
#     column_trans.fit_transform(csv)
    
    data = csv.to_numpy()

    val_cumulative = {}
    frame_ids = set()
    for person in data:
        frame_ids.add(person[0])
        frame, person_id = person[:2]

        vel_array = []
        for vel in range(5, 70, 4):
            vel_array.append(vel)

        total_vel = sum(vel_array)

        if person_id in val_cumulative:
            val_cumulative[person_id] += total_vel
        else:
            val_cumulative[person_id] = 0
        
        
    val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
    top_people = dict(itertools.islice(val_cumulative.items(), person_count))
    top_people_dataset = {}
    
    current_frame = 0
    for frame_id in frame_ids:
        # Trim to 280 if above
        if current_frame == max_frames:
            break
        top_people_dataset[frame_id] = []
        people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
        fill_amount = person_count
        for person in people:
            top_people_dataset[frame_id].append(person[2:])
            fill_amount -= 1
        for i in range(0, fill_amount):
            top_people_dataset[frame_id].append(np.zeros(68))
        current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
                
    tmp = np.asarray(list(top_people_dataset.values()))
    
#     x_train.append(tmp)
    for entry in tmp:
        x_test.append(entry)

#      = np.append(x_train, np.asarray([list(top_people_dataset.values())]), axis=0)
        
#     print(x_train.shape)
    
    if violent == 0:
        for entry in np.zeros(len(tmp)):
            y_test.append(entry)
    else:
        for entry in np.ones(len(tmp)):
            y_test.append(entry)

    #print(model.test_on_batch(x_test, y_test))
# model.fit(x_train, y_train, epochs, batch_size=batchs, verbose=1)
# model.save("rnn.h5", overwrite=True)

x_test = np.asarray(list(x_test))
y_test = np.asarray(list(y_test))

# model.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
scores = model.evaluate(x_test, y_test, verbose=1)

5600/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [88]:
output.shape

(124, 5, 68)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])